# 1. 인공지능 중심 금융 예시

In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv('../../source/tr_eikon_eod_data.csv',
									 index_col = 0, parse_dates = True)
data = pd.DataFrame(data['AAPL.0'])                        # 1. 애플 주식(AAPL.0)의 과거 종가 데이터 선택
data['Returns'] = np.log(data / data.shift())              # 2. 전체 데이터의 로그 수익률 계산
data.dropna(inplace=True)

lags = 6

cols = []
for lag in range(1, lags + 1):
		col = 'lag_{}'.format(lag)
		data[col] = np.sign(data['Returns'].shift(lag))        # 3. 로그 수익률 데이터를 특정 방향(+1 또는 -1)으로 지연시킨 열을 포함한 데이터프레임을 만듦
		cols.append(col)
data.dropna(inplace=True)

In [ ]:
from sklearn.svm import SVC

model = SVC(gamma = 'auto')                               # 1. 모형 객체 생성
model.fit(data[cols], np.sign(data['Returns']))           # 2. 특정 데이터와 레이블 데이터(주가 방향)로 모형 학습
SVC(C = 1.0, cache_size = 200, class_weight = None, coef0 = 0.0,
		decision_function_shape = 'ovr', degree = 3, gamma = 'auto', kernel = 'rbf',
		max_iter = -1, probability = False, random_state = None, shrinking = True,
		tol = 0.001, verbose = False)

data['Prediction'] = model.predict(data[cols])            # 3. 학습된 모형으로 예측한 결과를 매매 전략에 적용

data['Strategy'] = data['Prediction'] * data['Returns']   # 4. 예측값으로 만든 매매 전략과 벤치마크 전략의 로그 수익률을 계산

data[['Returns', 'Strategy']].cumsum().apply(np.exp).plot(figsize = (10, 6));  # 5. ML 기반 매매 전략의 성과와 단순히 매수를 유지하는 벤치마크 투자의 성과를 그림으로 비교 
